In [87]:
from google.colab import files
uploaded = files.upload()

Saving dfaftersentimentpolaritytag.csv to dfaftersentimentpolaritytag (1).csv
Saving out.csv to out (4).csv
Saving test.csv to test (4).csv


In [2]:
!pip3 install flair
import re
#import emoji
import nltk
import en_core_web_sm
import pandas as pd
import numpy as np
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from textblob import TextBlob

nltk.download('vader_lexicon')
nltk.download('words')
words = set(nltk.corpus.words.words())
nlp = en_core_web_sm.load()

     |████████████████████████████████| 286kB 8.1MB/s 
     |████████████████████████████████| 1.2MB 14.0MB/s 
     |████████████████████████████████| 776.8MB 23kB/s 
     |████████████████████████████████| 798kB 46.3MB/s 
     |████████████████████████████████| 19.7MB 51.4MB/s 
     |████████████████████████████████| 983kB 35.9MB/s 
     |████████████████████████████████| 71kB 6.0MB/s 
     |████████████████████████████████| 2.0MB 25.3MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 3.2MB 43.2MB/s 
     |████████████████████████████████| 890kB 46.8MB/s 
  Created wheel for gdown: filename=gdown-3.12.2-cp37-none-any.whl size=9693 sha256=aa3f3b0e8e3aa6a8e7f4de264770daf6b3414cd8eca2a4321943c610fa87fcb6
  Stored in directory: /root/.cache/pip/wheels/81/d0/d7/d9983facc6f2775411803e0e2d30ebf98efbf2fc6e57701e09
Successfully built gdown
  Created wheel for sqlitedict: fil

/usr/local/lib/python3.7/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


In [3]:
import gensim
# upgrade gensim if you can't import softcossim
from gensim.matutils import softcossim 
from gensim import corpora
import gensim.downloader as api
from gensim.utils import simple_preprocess
print(gensim.__version__)
#> '3.6.0'

# Download the FastText model
fasttext_model300 = api.load('fasttext-wiki-news-subwords-300')

3.6.0
[==================================================] 100.0% 958.5/958.4MB downloaded


## Accuracy testt

In [88]:
import io
df = pd.read_csv(io.BytesIO(uploaded['out.csv']))
df2 = pd.read_csv(io.BytesIO(uploaded['test.csv']))

In [89]:
def cat(x):
    if  x < 85 :
        return "1"
    if x < 90 and x >= 85:
        return "2"
    if x < 95 and x>= 90:
        return "3"
    if x>=95:
        return "4"


df2['category'] = df2['points'].apply(lambda x: cat(x))

In [90]:
df2.head()

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,variety,winery,category
0,0,US,This tremendous 100% varietal wine hails from ...,Martha's Vineyard,96,235.0,California,Napa Valley,Napa,Cabernet Sauvignon,Heitz,4
1,1,Spain,"Ripe aromas of fig, blackberry and cassis are ...",Carodorum Selección Especial Reserva,96,110.0,Northern Spain,Toro,NaN,Tinta de Toro,Bodega Carmen Rodríguez,4
2,2,US,Mac Watson honors the memory of a wine once ma...,Special Selected Late Harvest,96,90.0,California,Knights Valley,Sonoma,Sauvignon Blanc,Macauley,4
3,3,US,"This spent 20 months in 30% new French oak, an...",Reserve,96,65.0,Oregon,Willamette Valley,Willamette Valley,Pinot Noir,Ponzi,4
4,4,France,"This is the top wine from La Bégude, named aft...",La Brûlade,95,66.0,Provence,Bandol,NaN,Provence red blend,Domaine de la Bégude,4


In [91]:
def cleaner(comment):
    comment = re.sub("@[A-Za-z0-9]+","",comment) #Remove @ sign
    comment = re.sub(r"(?:\@|http?\://|https?\://|www)\S+", "", comment) #Remove http links
    comment = re.sub(r'[^\w\s]','',comment)
    comment = comment.lower()
    comment = re.sub(r'\d+', '',comment) #remove numbers
    comment = " ".join(comment.split())
    #comment = ''.join(c for c in comment if c not in emoji.UNICODE_EMOJI) #Remove Emojis
    comment = comment.replace("#", "").replace("_", " ") #Remove hashtag sign but keep the text
    comment = " ".join(w for w in nltk.wordpunct_tokenize(comment) 
         if w.lower() in words or not w.isalpha())
    return comment

In [30]:
def lemmatizer(text):        
    sent = []
    doc = nlp(text)
    for word in doc:
        sent.append(word.lemma_)
    return " ".join(sent)

In [92]:
df2['processed_description'] = df2['description'].map(lambda x: cleaner(x))
df2['lemmatised_content'] = df2['processed_description'].apply(lambda x: lemmatizer(x))
df2['lemmatised_content']=df2['lemmatised_content'].str.replace('-PRON-', '')
df2['stemming_content'] = df2['lemmatised_content']

In [93]:
from nltk.stem.porter import *
stemmer = PorterStemmer()

def stem_sentences(sentence):
    tokens = sentence.split()
    stemmed_tokens = [stemmer.stem(token) for token in tokens]
    return ' '.join(stemmed_tokens)

df2['final_content'] = df2['stemming_content'].apply(stem_sentences)

In [94]:

import flair
from flair.models import TextClassifier
from segtok.segmenter import split_single
from flair.data import Sentence

def make_sentences(text):
    """ Break apart text into a list of sentences """
    sentences = [sent for sent in split_single(text)]
    return sentences

def predict(sentence):
    """ Predict the sentiment of a sentence """
    if sentence == "":
        return 0
    text = Sentence(sentence)
    # stacked_embeddings.embed(text)
    classifier.predict(text)
    value = text.labels[0].to_dict()['value'] 
    if value == 'POSITIVE':
        result = text.to_dict()['labels'][0]['confidence']
    else:
        result = -(text.to_dict()['labels'][0]['confidence'])
    return round(result, 3)

def get_scores(sentences):
    """ Call predict on every sentence of a text """
    results = []
    for i in range(0, len(sentences)): 
        results.append(predict(sentences[i]))
    return results

In [95]:
#flair
classifier = TextClassifier.load('en-sentiment')

df2['sentences'] = df2.final_content.apply(make_sentences)
df2['flair_scores'] = df2.sentences.apply(get_scores)
df2 = df2.drop(columns='sentences')
df2['flair_scores'] = [','.join(map(str, l)) for l in df2['flair_scores']]
df2["flair_scores"] = pd.to_numeric(df2["flair_scores"], downcast="float")
conditions = [
(df2['flair_scores'] >= 0.05),
(df2['flair_scores'] <= -0.05),
((df2['flair_scores'] < 0.05) & (df2['flair_scores'] > -0.05))
]

values = ['positive', 'negative', 'neutral']

df2['flair_sentiment'] = np.select(conditions, values)

2021-04-03 11:04:47,014 loading file /root/.flair/models/sentiment-en-mix-distillbert_4.pt


In [96]:
#vader

sentiment_analyser = SentimentIntensityAnalyzer()

sentiments_vader = []

for i in df2.final_content:
    sentiments_vader.append(sentiment_analyser.polarity_scores(i).get('compound'))
    
sentiments_vader = np.asarray(sentiments_vader)

df2['sentiment_vader'] = pd.Series(data=sentiments_vader)
df2 = df2.reset_index(drop=True)

conditions = [
    (df2['sentiment_vader'] >= 0.05),
     (df2['sentiment_vader'] <= -0.05),
    ((df2['sentiment_vader'] < 0.05) & (df2['sentiment_vader'] > -0.05))
    ]

values = ['positive', 'negative', 'neutral']

df2['vader'] = np.select(conditions, values)


In [97]:
#textblob

sentiments_textblob = []

for review in df2.final_content:
    review = TextBlob(review)
    sentiments_textblob.append(review.sentiment.polarity)

sentiments_textblob = np.asarray(sentiments_textblob)   

df2['sentiment_textblob'] = pd.Series(data=sentiments_textblob)
df2= df2.reset_index(drop=True)
conditions = [
    (df2['sentiment_textblob'] > 0),
    (df2['sentiment_textblob'] < 0),
    (df2['sentiment_textblob'] == 0)
    ]

values = ['positive', 'negative', 'neutral']

df2['textblob'] = np.select(conditions, values)


In [98]:
#final sentiment
df2['final_sentiment']=''
for data in range(0,len(df2)):
    lst1 = []
    flair = df2['flair_sentiment'][data]
    vader = df2['vader'][data]
    textblob = df2['textblob'][data]
    lst1.append(flair)
    if vader in lst1:
        df2['final_sentiment'][data] = vader
    else:
        lst1.append(vader)
        if textblob in lst1:
            df2['final_sentiment'][data] = textblob
        else:
            df2['final_sentiment'][data] = ('unknown')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [99]:
cat1_pos=df['Words'][0]
cat1_neg=df['Words'][1]
cat2_pos=df['Words'][2]
cat2_neg=df['Words'][3]
cat3_pos=df['Words'][4]
cat3_neg=df['Words'][5]
cat4_pos=df['Words'][6]
cat4_neg=df['Words'][7]

In [100]:
documents = [cat1_pos, cat1_neg, cat2_pos, cat2_neg, cat3_pos, cat3_neg, cat4_pos, cat4_neg]


In [101]:
df2['soft_consine_score']=''
# Prepare a dictionary and a corpus.
dictionary = corpora.Dictionary([simple_preprocess(doc) for doc in documents])

# Prepare the similarity matrix
similarity_matrix = fasttext_model300.similarity_matrix(dictionary, tfidf=None, threshold=0.0, exponent=2.0, nonzero_limit=100)

# Convert the sentences into bag-of-words vectors.
cat1_pos_1 = dictionary.doc2bow(simple_preprocess(cat1_pos))
cat1_neg_1  = dictionary.doc2bow(simple_preprocess(cat1_neg))
cat2_pos_1  = dictionary.doc2bow(simple_preprocess(cat2_pos))
cat2_neg_1  = dictionary.doc2bow(simple_preprocess(cat2_neg))
cat3_pos_1  = dictionary.doc2bow(simple_preprocess(cat3_pos))
cat3_neg_1  = dictionary.doc2bow(simple_preprocess(cat3_neg))
cat4_pos_1  = dictionary.doc2bow(simple_preprocess(cat4_pos))
cat4_neg_1  = dictionary.doc2bow(simple_preprocess(cat4_neg))
for i in range(0,len(df2['final_content'])):
  df2['soft_consine_score'][i]=dictionary.doc2bow(simple_preprocess(df2['final_content'][i]))

sentences = [cat1_pos_1, cat1_neg_1, cat2_pos_1, cat2_neg_1, cat3_pos_1, cat3_neg_1, cat4_pos_1, cat4_neg_1]
sentences_name = ['cat1_pos_1', 'cat1_neg_1', 'cat2_pos_1', 'cat2_neg_1', 'cat3_pos_1', 'cat3_neg_1', 'cat4_pos_1', 'cat4_neg_1']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [102]:
df2['soft_consine_score_final']=''
output=[]
max_list=[]
for x in range(0,len(df2['final_content'])):
  for i in range(0,len(sentences)):
    output.append(softcossim(df2['soft_consine_score'][x], sentences[i], similarity_matrix))
  a=output.index(max(output))
  max_list.append(str(output[a])+','+sentences_name[a])
  df2['soft_consine_score_final'][x]=sentences_name[a]
  output=[]
  
print(max_list)
#a=output.index(max(output))
#print (str(output[a])+","+ sentences_name[a])

Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if _

['0.5189714744043722,cat2_neg_1', '0.0,cat1_pos_1', '0.5408762575182594,cat3_pos_1', '0.5445005409861997,cat4_pos_1', '0.42362414713206686,cat3_pos_1', '0.4487779627116087,cat3_pos_1', '0.620454260804449,cat2_neg_1', '0.6286304046071005,cat1_neg_1', '0.2744339238473288,cat4_pos_1', '0.44644812617240726,cat1_pos_1', '0.723041292317925,cat3_pos_1', '0.5112462520354815,cat4_pos_1', '0.5324872929587796,cat3_neg_1', '0.0,cat1_pos_1', '0.645764177154172,cat1_pos_1', '0.0,cat1_pos_1', '0.6130308314026347,cat4_neg_1', '0.5644562971831195,cat4_pos_1', '0.793270630253647,cat4_pos_1', '0.0,cat1_pos_1', '0.4981506584857558,cat1_pos_1', '0.628455780722937,cat4_pos_1', '0.7326815050189063,cat3_neg_1', '0.6441224913854473,cat4_pos_1', '0.489628524845793,cat2_neg_1', '0.623973734321552,cat3_pos_1', '0.6368446126069242,cat4_pos_1', '0.5445005409861997,cat4_pos_1', '0.4813927010621736,cat3_neg_1', '0.5716666246745131,cat4_neg_1', '0.3114033559132057,cat1_neg_1', '0.6274925093179646,cat3_pos_1', '0.48408

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/usr/local/lib/python3.7/dis

In [103]:
df2['predict_category']=''
for i in range(0,len(df2['soft_consine_score_final'])):
  if 'cat1' in df2['soft_consine_score_final'][i]:
    df2['predict_category'][i]='1'
  if 'cat2' in df2['soft_consine_score_final'][i]:
    df2['predict_category'][i]='2'
  if 'cat3' in df2['soft_consine_score_final'][i]:
    df2['predict_category'][i]='3'
  if 'cat4' in df2['soft_consine_score_final'][i]:
    df2['predict_category'][i]='4'

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: Sett

In [104]:
df2.head()

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,variety,winery,category,processed_description,lemmatised_content,stemming_content,final_content,flair_scores,flair_sentiment,sentiment_vader,vader,sentiment_textblob,textblob,final_sentiment,soft_consine_score,soft_consine_score_final,predict_category
0,0,US,This tremendous 100% varietal wine hails from ...,Martha's Vineyard,96,235.0,California,Napa Valley,Napa,Cabernet Sauvignon,Heitz,4,this tremendous varietal wine from and was age...,this tremendous varietal wine from and be age ...,this tremendous varietal wine from and be age ...,thi tremend variet wine from and be age over t...,1.000,positive,0.8750,positive,0.272222,positive,positive,"[(45, 1), (78, 1)]",cat2_neg_1,2
1,1,Spain,"Ripe aromas of fig, blackberry and cassis are ...",Carodorum Selección Especial Reserva,96,110.0,Northern Spain,Toro,NaN,Tinta de Toro,Bodega Carmen Rodríguez,4,ripe of fig blackberry and cassis are and by a...,ripe of fig blackberry and cassis be and by a ...,ripe of fig blackberry and cassis be and by a ...,ripe of fig blackberri and cassi be and by a o...,0.999,positive,0.7845,positive,0.364583,positive,positive,[],cat1_pos_1,1
2,2,US,Mac Watson honors the memory of a wine once ma...,Special Selected Late Harvest,96,90.0,California,Knights Valley,Sonoma,Sauvignon Blanc,Macauley,4,mac the memory of a wine once made by his moth...,mac the memory of a wine once make by mother ...,mac the memory of a wine once make by mother ...,mac the memori of a wine onc make by mother in...,0.943,positive,0.0000,neutral,-0.150000,negative,unknown,"[(56, 1), (62, 1)]",cat3_pos_1,3
3,3,US,"This spent 20 months in 30% new French oak, an...",Reserve,96,65.0,Oregon,Willamette Valley,Willamette Valley,Pinot Noir,Ponzi,4,this spent in new oak and fruit from aurora an...,this spend in new oak and fruit from aurora an...,this spend in new oak and fruit from aurora an...,thi spend in new oak and fruit from aurora and...,1.000,positive,0.2023,positive,0.059091,positive,positive,"[(55, 1)]",cat4_pos_1,4
4,4,France,"This is the top wine from La Bégude, named aft...",La Brûlade,95,66.0,Provence,Bandol,NaN,Provence red blend,Domaine de la Bégude,4,this is the top wine from la after the highest...,this be the top wine from la after the high po...,this be the top wine from la after the high po...,thi be the top wine from la after the high poi...,1.000,positive,0.6597,positive,0.175889,positive,positive,"[(48, 1), (73, 1)]",cat3_pos_1,3


In [105]:
df2['predict_sentiment']=''
for i in range(0,len(df2['soft_consine_score_final'])):
  if 'pos' in df2['soft_consine_score_final'][i]:
    df2['predict_sentiment'][i]='positive'
  if 'neg' in df2['soft_consine_score_final'][i]:
    df2['predict_sentiment'][i]='negative'

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [106]:
print(df2.category +', ' + df2.final_sentiment +', ' + df2.predict_category + ', ' + df2.predict_sentiment)

0       4, positive, 2, negative
1       4, positive, 1, positive
2        4, unknown, 3, positive
3       4, positive, 4, positive
4       4, positive, 3, positive
                  ...           
2935     3, unknown, 2, positive
2936    3, positive, 3, positive
2937    3, positive, 3, negative
2938    3, positive, 2, negative
2939     3, unknown, 3, negative
Length: 2940, dtype: object


In [107]:
from sklearn.metrics import accuracy_score

print(accuracy_score(df2['category'], df2['predict_category']))
print(accuracy_score(df2['final_sentiment'], df2['predict_sentiment']))

0.2938775510204082
0.5523809523809524


## user case1 filter

In [108]:
df3= pd.read_csv(io.BytesIO(uploaded['dfaftersentimentpolaritytag.csv']))

In [124]:
final_polarity = input("Please enter the polarity of the wine review. [positive/negative] :")
final_category = input("Please enter the category of the wine. [1/2/3/4] :")
final_country = input("Please enter the country of the wine review. [US/Italy/Spain/France/Germany/Argentina/etc] :")
final_score = input("Please enter your budget. [50,77,99] :")

result = df3.loc[np.logical_and(df3['final_sentiment'] == str(final_polarity) , df3['category'] == int(final_category))]
result = result.loc[np.logical_and(df3['country'] == str(final_country) , df3['price'] <= int(final_score))]
result

Please enter the polarity of the wine review. [positive/negative] :positive
Please enter the category of the wine. [1/2/3/4] :3
Please enter the country of the wine review. [US/Italy/Spain/France/Germany/Argentina/etc] :US
Please enter your budget. [50,77,99] :85


,Unnamed: 0,Unnamed: 0.1,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery,category,processed_description,lemmatised_content,final_content,flair_scores,flair_sentiment,sentiment_vader,vader,sentiment_textblob,textblob,final_sentiment
121,121,121,US,The vineyard is one of the better Chardonnay s...,Stuhlmuller Vineyard,92,36.0,California,Alexander Valley,Sonoma,NaN,NaN,Matrix 2007 Stuhlmuller Vineyard Chardonnay (A...,Chardonnay,Matrix,3,the vineyard is one of the better in valley an...,the vineyard be one of the well in valley and ...,the vineyard be one of the well in valley and ...,0.996,positive,0.7876,positive,2.189394e-01,positive,positive
122,122,122,US,Defines Rockpile Zinfandel in intensity of fru...,Rockpile Cemetary Vineyard,92,39.0,California,Rockpile,Sonoma,NaN,NaN,Mauritson 2007 Rockpile Cemetary Vineyard Zinf...,Zinfandel,Mauritson,3,zinfandel in intensity of fruit and high alcoh...,zinfandel in intensity of fruit and high alcoh...,zinfandel in intensity of fruit and high alcoh...,0.989,positive,0.3400,positive,9.904762e-02,positive,positive
124,124,124,US,This sophisticated wine is mostly Cabernet Sau...,NaN,92,45.0,California,Napa Valley,Napa,NaN,NaN,Silverado 2006 Cabernet Sauvignon (Napa Valley),Cabernet Sauvignon,Silverado,3,this sophisticated wine is mostly cabernet whi...,this sophisticated wine be mostly cabernet whi...,this sophisticated wine be mostly cabernet whi...,0.999,positive,0.9451,positive,3.031250e-01,positive,positive
134,134,134,US,Give this young Cab time in the cellar to come...,NaN,91,78.0,California,Napa Valley,Napa,NaN,NaN,Clark-Clauden 2007 Cabernet Sauvignon (Napa Va...,Cabernet Sauvignon,Clark-Clauden,3,give this young cab time in the cellar to come...,give this young cab time in the cellar to come...,give this young cab time in the cellar to come...,1.000,positive,0.6486,positive,1.622449e-01,positive,positive
142,142,142,US,"A beautiful wine, sleek, crisp and ultra-dry, ...",NaN,90,17.0,California,Santa Lucia Highlands,Central Coast,NaN,NaN,Banyan 2007 Riesling (Santa Lucia Highlands),Riesling,Banyan,3,a beautiful wine sleek crisp and with the most...,a beautiful wine sleek crisp and with the most...,a beautiful wine sleek crisp and with the most...,1.000,positive,0.5994,positive,2.093333e-01,positive,positive
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129945,129945,129945,US,Hailing from one of the more popular vineyards...,Jurassic Park Vineyard Old Vines,90,20.0,California,Santa Ynez Valley,Central Coast,Matt Kettmann,@mattkettmann,Birichino 2013 Jurassic Park Vineyard Old Vine...,Chenin Blanc,Birichino,3,from one of the more popular for this varietal...,from one of the more popular for this varietal...,from one of the more popular for this varietal...,0.920,positive,-0.6327,negative,1.458333e-01,positive,positive
129949,129949,129949,US,There's no bones about the use of oak in this ...,Barrel Fermented,90,35.0,California,Napa Valley,Napa,Virginie Boone,@vboone,Flora Springs 2013 Barrel Fermented Chardonnay...,Chardonnay,Flora Springs,3,theres no about the use of oak in this wine wh...,there s no about the use of oak in this wine w...,there s no about the use of oak in this wine w...,0.999,positive,0.5994,positive,4.500000e-01,positive,positive
129950,129950,129950,US,This opens with herbaceous dollops of thyme an...,Blocks 7 & 22,90,35.0,California,Napa Valley,Napa,Virginie Boone,@vboone,Hendry 2012 Blocks 7 & 22 Zinfandel (Napa Valley),Zinfandel,Hendry,3,this with herbaceous of thyme and earth and sl...,this with herbaceous of thyme and earth and sl...,this with herbaceous of thyme and earth and sl...,0.988,positive,0.0000,positive,-1.222222e-01,negative,positive
129952,129952,129952,US,This Zinfandel from the eastern section of Nap...,NaN,90,22.0,California,Chiles Valley,Napa,Virginie Boone,@vboone,Houdini 2011 Zinfandel (Chiles Valley),

In [125]:
final_result = result.drop(columns=['Unnamed: 0','designation','province','region_1','region_2','taster_name'
                                   ,'taster_twitter_handle','lemmatised_content','final_content',
                                   'winery','processed_description','flair_scores','flair_sentiment',
                                  'sentiment_vader','vader','sentiment_textblob','textblob'])

In [126]:
final_result

,Unnamed: 0.1,country,description,points,price,title,variety,category,final_sentiment
121,121,US,The vineyard is one of the better Chardonnay s...,92,36.0,Matrix 2007 Stuhlmuller Vineyard Chardonnay (A...,Chardonnay,3,positive
122,122,US,Defines Rockpile Zinfandel in intensity of fru...,92,39.0,Mauritson 2007 Rockpile Cemetary Vineyard Zinf...,Zinfandel,3,positive
124,124,US,This sophisticated wine is mostly Cabernet Sau...,92,45.0,Silverado 2006 Cabernet Sauvignon (Napa Valley),Cabernet Sauvignon,3,positive
134,134,US,Give this young Cab time in the cellar to come...,91,78.0,Clark-Clauden 2007 Cabernet Sauvignon (Napa Va...,Cabernet Sauvignon,3,positive
142,142,US,"A beautiful wine, sleek, crisp and ultra-dry, ...",90,17.0,Banyan 2007 Riesling (Santa Lucia Highlands),Riesling,3,positive
...,...,...,...,...,...,...,...,...,...
129945,129945,US,Hailing from one of the more popular vineyards...,90,20.0,Birichino 2013 Jurassic Park Vineyard Old Vine...,Chenin Blanc,3,positive
129949,129949,US,There's no bones about the use of oak in this ...,90,35.0,Flora Springs 2013 Barrel Fermented Chardonnay...,Chardonnay,3,positive
129950,129950,US,This opens with herbaceous dollops of thyme an...,90,35.0,Hendry 2012 Blocks 7 & 22 Zinfandel (Napa Valley),Zinfandel,3,positive
129952,129952,US,This Zinfandel from the eastern section of Nap...,90,22.0,Houdini 2011 Zinfandel (Chiles Valley),Zinfandel,3,positive


## User input 


In [71]:
test= input("Please enter a review: ")

Please enter a review: This wine is in peak condition. The tannins and the secondary flavors dominate this ripe leather-textured wine. The fruit is all there as well: dried berries and hints of black-plum skins. It is a major wine right at the point of drinking with both the mature flavors and the fruit in the right balance.


In [72]:
test=cleaner(test)
test=lemmatizer(test)


In [75]:
test

[(55, 1)]

In [74]:
test  = dictionary.doc2bow(simple_preprocess(test))

In [78]:
usertest_output=[]
for i in sentences:
    usertest_output.append(softcossim(test, i, similarity_matrix))

  
a=usertest_output.index(max(usertest_output))
print(usertest_output[a],sentences_name[a])

0.4734077177831222 cat4_pos_1


In [118]:
usertest_output

[0.32006908569846015,
 0.3198115634894101,
 0.30724993680658047,
 0.3477672486240593,
 0.2959012866478315,
 0.31407314560031757,
 0.4734077177831222,
 0.4538423910978828]

In [ ]:
#https://www.machinelearningplus.com/nlp/cosine-similarity/